# Pre-trained model test with InceptionV3

In [1]:
# import libraries
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions, InceptionV3

In [2]:
# load and preprocess image
image = load_img("Biscotte.jpg", target_size = (299, 299, 3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

In [3]:
# load and fit model with Biscotte picture
model = InceptionV3()
yhat = model.predict(image)
label = decode_predictions(yhat)
label = label[0][0]

96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


In [5]:
# show result
print("%s (%.2f%%)" % (label[1], label[2]*100))

English_springer (91.17%)


In [6]:
# with Ramses picture...
image = load_img("Ramses.jpg", target_size = (299, 299, 3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
yhat = model.predict(image)
label = decode_predictions(yhat)
label = label[0][0]
print("%s (%.2f%%)" % (label[1], label[2]*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
cloak (23.36%)


not good...

In [8]:
# with Bouli picture...
image = load_img("Bouli.JPEG", target_size = (299, 299, 3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
yhat = model.predict(image)
label = decode_predictions(yhat)
label = label[0][0]
print("%s (%.2f%%)" % (label[1], label[2]*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Border_collie (47.44%)


In [9]:
# with another Bouli picture...
image = load_img("Bouli2.JPEG", target_size = (299, 299, 3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
yhat = model.predict(image)
label = decode_predictions(yhat)
label = label[0][0]
print("%s (%.2f%%)" % (label[1], label[2]*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Border_collie (69.17%)


# With EfficientNetV2S

In [39]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input, decode_predictions, EfficientNetV2S

In [40]:
model = EfficientNetV2S()

87549136/87549136 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


In [42]:
# with Ramses picture...
image = load_img("Ramses.jpg", target_size = (384, 384, 3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
yhat = model.predict(image)
label = decode_predictions(yhat)
label = label[0][0]
print("%s (%.2f%%)" % (label[1], label[2]*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
bloodhound (34.54%)


---
# CUSTOMIZATION OF A PRE-TRAINED MODEL
---

In [52]:
# import libraries
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

In [44]:
# load data
photos = np.load("Cats_Dogs_photos.npy")
labels = np.load("Cats_Dogs_labels.npy")

In [60]:
# specify training and validation data
X_train = photos[:1600, :, :]
X_test = photos[1600:, :, :]
y_train = labels[:1600]
y_test = labels[1600:]

In [54]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1600, 192, 192, 3),
 (400, 192, 192, 3),
 (1600, 192, 192, 3),
 (400, 192, 192, 3))

In [55]:
# pre-processing input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [56]:
# load the convolutional base model
base_model = MobileNetV2(include_top = False, input_shape = (192, 192, 3))

In [57]:
# set layers as not trainable
for layer in base_model.layers :
    layer.trainable = False

In [58]:
# add new classifier layers
x = Flatten()(base_model.layers[-1].output)
x = Dense(128, activation = "relu", kernel_initializer = "he_uniform")(x)
output = Dense(1, activation = "sigmoid")(x)

In [61]:
# define new model, compile and fit
model = Model(inputs = base_model.inputs, outputs = output)
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(X_train, y_train, epochs = 4, batch_size = 64, validation_data = (X_test, y_test))

Epoch 1/4
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.4553 - loss: 9.9186 - val_accuracy: 0.5300 - val_loss: 0.7946
Epoch 2/4
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 998ms/step - accuracy: 0.5661 - loss: 0.7429 - val_accuracy: 0.6375 - val_loss: 0.7033
Epoch 3/4
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 979ms/step - accuracy: 0.6380 - loss: 0.6379 - val_accuracy: 0.6750 - val_loss: 0.5944
Epoch 4/4
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 970ms/step - accuracy: 0.6807 - loss: 0.5799 - val_accuracy: 0.6825 - val_loss: 0.5817


In [63]:
image = load_img("Bouli.JPEG", target_size = (192, 192, 3))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
yhat = model.predict(image)
yhat

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step


array([[0.18542042]], dtype=float32)

---
# EXERCICE
---

In [85]:
# import libraries
from keras import Input
from keras import layers
from keras.utils import plot_model

In [108]:
# building model function
n_input = 2

def create_model() :
    x0 = [Input(shape = (192, 192, 3, 1)) for _ in range(n_input)]
    x1 = [layers.Conv3D(8, (3, 3, 1), strides = (1, 1, 1), activation = "relu", input_shape = (192, 192, 3, 1))(x) for x in x0]
    x2 = [layers.MaxPooling3D((2, 2, 1))(x) for x in x1]
    
    x3 = [layers.Concatenate()([x2[i], x2[i + 1]]) for i in range(0, len(x2), 2)]
    x4 = [layers.Conv3D(12, (3, 3, 1), strides = (1, 1, 1), activation = "relu")(x) for x in x3]
    x5 = [layers.MaxPooling3D((2, 2, 1))(x) for x in x4]
    
    x6 = layers.Concatenate()(x5)
    x7 = layers.Flatten()(x6)
    x8 = layers.Dense(8, activation = "relu")(x7)
    x9 = layers.Dense(1, activation = "sigmoid")(x8)
    
    model = Model(inputs = x0, outputs = x9)
    
    plot_model(model, show_shapes = True, to_file = "model_graph_test.png")
    
    return model

In [110]:
model = create_model()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit([X_train for _ in range (n_input)], y_train, epochs = 15, batch_size = 64,
          validation_data = ([X_test for _ in range (n_input)], y_test))

Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 793ms/step - accuracy: 0.4801 - loss: 0.8545 - val_accuracy: 0.4700 - val_loss: 0.6932
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 636ms/step - accuracy: 0.5236 - loss: 0.6931 - val_accuracy: 0.4700 - val_loss: 0.6933
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 613ms/step - accuracy: 0.5151 - loss: 0.6931 - val_accuracy: 0.4700 - val_loss: 0.6934
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 611ms/step - accuracy: 0.5117 - loss: 0.6931 - val_accuracy: 0.4700 - val_loss: 0.6935
Epoch 5/15
 1/25 ━━━━━━━━━━━━━━━━━━━━ 16s 692ms/step - accuracy: 0.5156 - loss: 0.6930

KeyboardInterrupt: 